In [ ]:
# required for importin modules from other directories
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 


import numpy as np
#from KNNRegressor import KNNRegressor
import matplotlib.pyplot as plt

import config
from common import DataParser
from KNN.KNNRegressor import KNN_DEBUG, KNNRegressor

# House Prices Test

In [ ]:
x_train, y_train = DataParser.parse_test_housePrices(splitData=True)

k = 3
p = 1
weights = "uniform" #"distance"
reg = KNNRegressor(k=k, p=p)
reg.fit(x_train, y_train)

display(reg)

y_pred, distances = reg.predict(x_train.iloc[0:3,:], ret_distances=2)
y_pred

# Moneyball

In [ ]:
df = DataParser.parse_moneyball(splitData=False)

In [ ]:
numeric_features = ["RS", "RA", "OBP", "SLG", "BA", "Playoffs", "G", "OOBP", "OSLG"]

In [ ]:
df = df[numeric_features + ["W"]]
df.dropna(inplace=True)
x = df[numeric_features]
y = df["W"]
x, y

In [ ]:
def split_XY(x, y):
    x_train, x_split, y_train, y_split = train_test_split(x, y, test_size=0.4)
    x_valid, x_test, y_valid, y_test = train_test_split(x_split, y_split, test_size=0.5)
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def split_df(df):
    train, split = train_test_split(df, test_size=0.4)
    valid, test = train_test_split(split, test_size=0.5)
    return y_train, y_valid, y_test

def preprocess(df, ret_df=True):
    numeric_features = ["RS", "RA", "OBP", "SLG", "BA", "Playoffs", "G", "OOBP", "OSLG"]
    df = df[numeric_features + ["W"]]
    df.dropna(inplace=True)
    x = df[numeric_features]
    y = df["W"]
    return x, y, df

In [ ]:
dc = {"1": 1, "2":3}

for n, v in dc.items():
    print(f"n: {n} --> {v}")


In [ ]:
from time import time_ns as timer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor as skKNN
from sklearn.model_selection import train_test_split
import pandas as pd


COLS = ["name", "param", "r2", "mse", "train_time", "inference_time"]

def experiment(df, models):
    x, y, df = preprocess(df, ret_df=True)
    x_train, x_valid, x_test, y_train, y_valid, y_test = split_XY(x, y)

    results = pd.DataFrame(columns=COLS)

    for name, model in models.items():
        params = model.get_params()
        print(name)
        display(model)
        
        train_time = timer()
        model.fit(x_train, y_train)
        train_time = timer() - train_time

        inference_time = timer()
        predictions = model.predict(x_valid)
        inference_time = timer() - inference_time

        r2 = r2_score(y_valid, predictions)
        mse = mean_squared_error(y_valid, predictions)

        result = {
            "name": name,
            "param": params["k"],
            "r2": r2,
            "mse": mse,
            "train_time": train_time,
            "inference_time": inference_time
        }
        results.append(result, ignore_index=True)

    return results

In [ ]:
df = DataParser.parse_moneyball(splitData=False)

df

In [ ]:
models = []

k = 5
p = 2
reg = KNNRegressor() #(k=k, p=p)

knn = skKNN()
knn.get_params()

models = [reg, knn]
for x in models:
    display(str(x)+": ")
    display(x.get_params())
    print("_"*30)

models = {"my_knn": reg, "sk_knn": knn}

In [ ]:
results = experiment(df, models)

In [ ]:
k = 2
p = 1
reg = KNNRegressor(k=k, p=p)
reg.fit(x_train, y_train)

display(reg)

y_pred, distances = reg.predict(x_test, ret_distances=2)
y_pred

In [ ]:
distances


In [ ]:
np.mean(distances[:2,:4], axis=1)